# Part I - Exploring Prosper Loan Data
## by Israel Ogunmola

## Introduction
---

If you want to consolidate debt, finance a large purchase, or cover an emergency expense, a personal loan can be immensely useful. However, many personal loan lenders require borrowers to have good or excellent credit, making it difficult to qualify for a loan. Borrowers with credit ratings in the fair range or below may have better chances of obtaining loans at a better rate by working with a peer-to-peer lender.

[Prosper is a personal loan pioneer](https://www.prosper.com/) — the US company became the first firm to enter the peer-to-peer lending arena when it launched in 2005. Since then, the platfom has originated more than 20 billion USD in personal loans by matching over 1,170,000 borrowers to potential investors through its online platform. Prosper offers unsecured personal loans to customers who have a minimum credit score of 640. It also provides home equity lines of credit (HELOCs).

Our goal is to explore a sample of Prosper loan data to uncover borrower motivations when applying for loans, and identify several factors that may influence loan favorability.

## Importing Libraries
---
 
A great way to start is by importing the libraries and packages we need. We will import the **Numpy** and **Pandas** libraries to help us load and perform quick, vectorized operations on our data, then the **Matplotlib** and **Seaborn** libraries to help us build informing visuals:

In [1]:
# Data analysis and visualization packages
import numpy as np
import pandas as pd
import requests
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import seaborn as sns

# Configure visualization behaviours
%matplotlib inline
# %config InlineBackend.figure_format = "retina"

## Preliminary Wrangling
---
We will start by importing our dataset, `prosperLoanData.csv`, then reading it into a pandas dataframe:

In [2]:
df = pd.read_csv('./prosperLoanData.csv')

In [3]:
# Display quick summary information about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ListingKey                           113937 non-null  object 
 1   ListingNumber                        113937 non-null  int64  
 2   ListingCreationDate                  113937 non-null  object 
 3   CreditGrade                          28953 non-null   object 
 4   Term                                 113937 non-null  int64  
 5   LoanStatus                           113937 non-null  object 
 6   ClosedDate                           55089 non-null   object 
 7   BorrowerAPR                          113912 non-null  float64
 8   BorrowerRate                         113937 non-null  float64
 9   LenderYield                          113937 non-null  float64
 10  EstimatedEffectiveYield              84853 non-null   float64
 11  EstimatedLoss

### Initial Notes on Dataset Structure:
>- The dataframe comprises **113,937** rows and **81** columns (features). 56 of these 81 columns (69%) contain numeric data. There is a wealth of information that describes the situations surrounding each loan in the dataset.

### Features of Interest
The dataset currently contains loads of information. However, the goal of this exploration is to understand the different borrower motivations when applying for loans, including the different factors that may influence loan favorability. As a result, we will direct our exploratory efforts towards the following features:
>(1.) **ListingCreationDate:** The date the listing was created.

>(2.) **ListingCategory (numeric):** The category of the listing that the borrower selected when posting their listing: 0 - Not Available, 1 - Debt Consolidation, 2 - Home Improvement, 3 - Business, 4 - Personal Loan, 5 - Student Use, 6 - Auto, 7- Other, 8 - Baby&Adoption, 9 - Boat, 10 - Cosmetic Procedure, 11 - Engagement Ring, 12 - Green Loans, 13 - Household Expenses, 14 - Large Purchases, 15 - Medical/Dental, 16 - Motorcycle, 17 - RV, 18 - Taxes, 19 - Vacation, 20 - Wedding Loans.

>(3.) **BorrowerState:** The two letter abbreviation of the state of the address of the borrower at the time the Listing was created.

>(4.) **isBorrowerHomeowner:** A Borrower will be classified as a homowner if they have a mortgage on their credit profile or provide documentation confirming they are a homeowner.

>(5.) **IncomeRange:** The income range of the borrower at the time the listing was created.

>(6.) **IncomeVerifiable:** The borrower indicated they have the required documentation to support their income.

>(7.) **DebtToIncomeRatio:** The debt to income ratio of the borrower at the time the credit profile was pulled. This value is Null if the debt to income ratio is not available. This value is capped at 10.01 (any debt to income ratio larger than 1000% will be returned as 1001%).

>(8.) **StatedMonthlyIncome:** The monthly income the borrower stated at the time the listing was created.

>(9.) **ProsperRating (Alpha):** The Prosper Rating assigned at the time the listing was created between AA - HR. Applicable for loans originated after July 2009.

>(10.) **Term:** The length of the loan expressed in months.

>(11.) **EmploymentStatus:** The employment status of the borrower at the time they posted the listing.

>(12.) **LoanStatus:** The current status of the loan: Cancelled, Chargedoff, Completed, Current, Defaulted, FinalPaymentInProgress, PastDue. The PastDue status will be accompanied by a delinquency bucket.

>(13.) **LoanOriginalAmount:** The origination amount of the loan.

>(14.) **BorrowerAPR:** The Borrower's Annual Percentage Rate (APR) for the loan.

Generally, we aim to measure loan favorability in terms of **Prosper rating** and **Annual percentage rate (borrower APR)**.

## Data Assessment
---
We will now assess our dataframe, with particular focus on the features of interest. The major goal here is data exploration, hence findings on quality and tidiness will be collated under the assessment summary, and addressed using a brief and concise cleaning workflow.

First, let's create a list containing the key features for our analysis:

In [4]:
# Create a list of key feature names
key_features = ['ListingCreationDate', 'ListingCategory (numeric)', 'BorrowerState', 'IsBorrowerHomeowner',
                'IncomeRange', 'IncomeVerifiable', 'DebtToIncomeRatio', 'StatedMonthlyIncome', 'Term', 'ProsperRating (Alpha)',
                'EmploymentStatus', 'LoanStatus', 'LoanOriginalAmount', 'BorrowerAPR']

Next, we will examine a sample of records from these features in pandas, coupled with thorough visual assessments in a spreadsheet software like Ms Excel:

In [5]:
# Visually assess portions of the dataframe containing key features
df[key_features].sample(20)

,ListingCreationDate,ListingCategory (numeric),BorrowerState,IsBorrowerHomeowner,IncomeRange,IncomeVerifiable,DebtToIncomeRatio,StatedMonthlyIncome,Term,ProsperRating (Alpha),EmploymentStatus,LoanStatus,LoanOriginalAmount,BorrowerAPR
1046,2013-11-25 12:06:16.250000000,1,NH,True,"$25,000-49,999",True,0.31,3083.333333,36,A,Employed,Current,2000,0.15223
80149,2013-10-03 07:13:50.623000000,1,NJ,False,"$75,000-99,999",True,0.23,6666.666667,36,A,Employed,Current,9000,0.13799
90267,2013-11-04 20:52:23.437000000,1,CA,False,"$25,000-49,999",True,0.56,3916.666667,36,C,Employed,Current,10000,0.23898
46194,2013-07-24 08:06:44.670000000,1,IN,True,"$75,000-99,999",True,0.24,7750.000000,60,B,Employed,Current,18000,0.18136
79507,2008-06-14 05:14:15.757000000,4,NC,True,"$25,000-49,999",True,0.15,3666.666667,36,NaN,Full-time,Completed,6000,0.07680
19187,2013-04-09 11:24:48.690000000,1,GA,False,"$25,000-49,999",True,0.69,2333.333333,36,HR,Employed,Current,4000,0.35356
2867,2008-05-21 13:19:52.460000000,4,AZ,False,"$25,000-49,999",True,0.15,2600.000000,36,NaN,Full-time,Completed,2000,0.29233
81907,2012-04-17 12:52:45.140000000,13,MI,False,"$1-24,999",True,0.82,1000.000000,60,C,Employed,Current,13000,0.27462
7487,2014-02-27 13:13:26.180000000,1,NC,True,"$75,000-99,999",True,0.24,7807.416667,36,E,Employed,Current,7000,0.31196
55349,2007-07-24 16:06:45.743000000,0,CO,True,"$75,000-99,999",True,0.26,6267.000000,36,NaN,Full-time,Completed,4600,0.19991


Finally, we will check for duplicates, then preview a quick summary of these features, to get an idea of the overall distribution of datatypes, null values and the nomenclature of the column names:

In [6]:
# Examine for duplicates
duplicates = df[key_features].duplicated().sum()
print('There are {} duplicate records in the dataset'.format(duplicates))

There are 871 duplicate records in the dataset


In [7]:
df[key_features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ListingCreationDate        113937 non-null  object 
 1   ListingCategory (numeric)  113937 non-null  int64  
 2   BorrowerState              108422 non-null  object 
 3   IsBorrowerHomeowner        113937 non-null  bool   
 4   IncomeRange                113937 non-null  object 
 5   IncomeVerifiable           113937 non-null  bool   
 6   DebtToIncomeRatio          105383 non-null  float64
 7   StatedMonthlyIncome        113937 non-null  float64
 8   Term                       113937 non-null  int64  
 9   ProsperRating (Alpha)      84853 non-null   object 
 10  EmploymentStatus           111682 non-null  object 
 11  LoanStatus                 113937 non-null  object 
 12  LoanOriginalAmount         113937 non-null  int64  
 13  BorrowerAPR                11

### Assessment Summary
Visual and programmatic assessments, with regards to the features of interest, yeilded the following observations:
> 1. Key features need to be isolated from the dataset.
> 2. There are 871 duplicate records in the dataset.
> 3. **ListingCategory (numeric)** and **ProsperRating (Alpha)** can be reassigned with column names that are easier to work with.
> 4. **ListingCreationDate** is stored with the wrong datatype. It should be a pandas datetime object.
> 5. The numeric information in **ListingCategory (numeric)** could be better expanded to reflect the actual reasons for the loan. The data dictionary contains helpful information for this.
> 6. _Not employed_ entries in **IncomeRange** could be safely replaced with _0_.
> 7. Key features for loan favorability (**BorrowerAPR** and **ProsperRating (Alpha)**) contain null values. The **DebtToIncomeRatio** column also contains null values.
> 8. **ProsperRating** and **IncomeRange** are ordinal categorical variables and should be stored in ordered form.
> 9. Longitude and Latitude information can be collected from external sources to make **BorrowerState** visualizations easier.

## Data Cleaning
---
We will start by creating a copy of the original dataframe, then address each identified issue with brief cleaning workflows:

In [8]:
clean_df = df.copy()

#### 1. _Isolate key features from the dataframe_

In [9]:
# Filter out the key features from the original dataframe
clean_df = clean_df[key_features]

# Verify the changes made
assert len(clean_df.columns) == len(key_features)

#### 2. _Drop the duplicate records from the dataframe_

In [10]:
clean_df = clean_df.drop_duplicates()

# Verify results
assert not clean_df.duplicated().any()

#### 3. _Reassign ListingCategory (numeric) and ProsperRating (Alpha) with proper column names_

In [11]:
# Rename the columns
clean_df = clean_df.rename(columns = {'ListingCategory (numeric)': 'ListingCategory', 'ProsperRating (Alpha)': 'ProsperRating'})

# verify code results
for col_name in ['ListingCategory', 'ProsperRating']:
    assert col_name in clean_df.columns

#### 4. _Convert ListingCreationDate to DateTime object_

In [12]:
clean_df.ListingCreationDate = pd.to_datetime(df.ListingCreationDate)

# Verify results
clean_df.dtypes[0:2]

ListingCreationDate    datetime64[ns]
ListingCategory                 int64
dtype: object

#### 5. _Map the ListingCategory column to the right category titles_

In [13]:
# Store all the numeric categories and their titles in a dictionary
category_titles = {0 : 'Not Available', 1 : 'Debt Consolidation', 2 : 'Home Improvement', 3: 'Business', 
                   4 : 'Personal Loan', 5 : 'Student Use', 6 : 'Auto', 7 : 'Other', 8 : 'Baby & Adoption', 
                   9 : 'Boat', 10 : 'Cosmetic Procedure', 11 : 'Engagement Ring', 12 : 'Green Loans',
                  13 : 'Household Expenses', 14 : 'Large Purchases', 15 : 'Medical or Dental', 16 : 'Motorcycle',
                  17 : 'RV', 18 : 'Taxes', 19 : 'Vacation', 20 : 'Wedding Loans'}

# Map the dictionary contents to the ListingCategory column
clean_df.ListingCategory = clean_df.ListingCategory.map(category_titles)

# Preview results
clean_df.ListingCategory.unique()

array(['Not Available', 'Home Improvement', 'Motorcycle',
       'Debt Consolidation', 'Other', 'Household Expenses', 'Auto',
       'Medical or Dental', 'Wedding Loans', 'Vacation', 'Business',
       'Taxes', 'Baby & Adoption', 'Personal Loan', 'Engagement Ring',
       'Large Purchases', 'Student Use', 'Boat', 'RV',
       'Cosmetic Procedure', 'Green Loans'], dtype=object)

#### 6. _Replace 'Not employed' entries in IncomeRange with $0_

In [14]:
clean_df.IncomeRange = clean_df.IncomeRange.str.replace('Not employed', '$0')

# Verify changes
assert 'Not employed' not in clean_df.IncomeRange

#### 7. _Address the null values in BorrowerAPR, ProsperRating, DebtToIncomeRatio and EmploymentStatusDuration columns_
**BorrowerAPR** and **ProsperRating** are integral to our analysis. They are our dependent variables. Hence, it is important to maintain the integrity of their data. We will drop the null values if their combined sum is **lesser than 30%** of the records in our entire dataframe. This ensures that we have enough data to analyze even after the null values have been removed.

Values in the **DebtToIncomeRatio** column can be null if:
>- The borrower has never taken a debt on the platform before.
>- The borrower is currently not servicing any debt.
 
As a result, we will only be dealing with null values in the _BorrowerAPR and ProsperRating_ columns.

In [15]:
# Create a list of columns with null values
null_columns = ['BorrowerAPR', 'ProsperRating']

# Estimate the number of null records in each column with a list comprehension
null_counts = [clean_df[col].isnull().sum() for col in null_columns]

# Calculate the total proportion of null values in the dataframe
null_proportion = sum(null_counts) / df.shape[0]

print('The combined percentage of null records is ' + str(null_proportion * 100)[:4] + '%')

The combined percentage of null records is 25.5%


Since the combined percentage is lesser than 30%, we will proceed to drop the null records from our dataframe:

In [16]:
# Filter out null values from the dataframe
for col in null_columns:
    clean_df = clean_df[clean_df[col].notnull()]

# Verify results
for col in null_columns:
    assert clean_df[col].notnull().any()

#### 8. _Convert ProsperRating and IncomeRange columns to ordered categorical types_

In [17]:
# Store the correct variable orders in a dictionary
order_dict = {'ProsperRating': ['HR', 'E', 'D', 'C', 'B', 'A', 'AA'],
              'IncomeRange': ['$0', '$1-24,999', '$25,000-49,999', 
                              '$50,000-74,999', '$75,000-99,999', '$100,000+']}

# Assign each column to the proper order
for key, value in order_dict.items():
    correct_order = pd.api.types.CategoricalDtype(categories=value, ordered=True)
    clean_df[key] = clean_df[key].astype(correct_order)

# Verify changes
clean_df[order_dict.keys()].dtypes

ProsperRating    category
IncomeRange      category
dtype: object

## Gathering Additional Data
---
The information in the **BorrowerState** column is an abbreviation of the different states in the US, depending on the borrower location. To conveniently visualize and explore this kind of information, we will need to pull data from outside sources, especially one that contains the full name, latitude and longitude for each state.

A kaggle user [Washim Ahmed](https://www.kaggle.com/washimahmed), has already uploaded this information online. We will download [the csv file](https://www.kaggle.com/datasets/washimahmed/usa-latlong-for-state-abbreviations?resource=download&select=statelatlong.csv) programatically, using the requests library:

In [18]:
# Programmatically download the csv file
# url = 'https://cutt.ly/fLuimKG'
# response = requests.get(url)
# with open('state_data.csv', 'wb') as file:
#     file.write(response.content)

In [19]:
# Read-in the newly downloaded dataframe
state_df = pd.read_csv('./state_data.csv')
state_df.head(3)

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona


Next, we merge the state dataframe into our cleaned dataframe:

In [20]:
# Merge clean_df and state_df
loan_df = pd.merge(clean_df, state_df, left_on='BorrowerState', right_on='State')
loan_df.head(3)

,ListingCreationDate,ListingCategory,BorrowerState,IsBorrowerHomeowner,IncomeRange,IncomeVerifiable,DebtToIncomeRatio,StatedMonthlyIncome,Term,ProsperRating,EmploymentStatus,LoanStatus,LoanOriginalAmount,BorrowerAPR,State,Latitude,Longitude,City
0,2014-02-27 08:28:07.900,Home Improvement,CO,False,"$50,000-74,999",True,0.18,6125.000000,36,A,Employed,Current,10000,0.12016,CO,38.997934,-105.550567,Colorado
1,2010-06-16 16:23:44.533,Debt Consolidation,CO,True,"$75,000-99,999",True,0.26,6666.666667,36,A,Full-time,Completed,4000,0.11296,CO,38.997934,-105.550567,Colorado
2,2013-05-01 08:18:59.183,Debt Consolidation,CO,True,"$50,000-74,999",True,0.35,5416.666667,36,A,Employed,Current,15000,0.15833,CO,38.997934,-105.550567,Colorado


Finally, we will assign the full state name to the **BorrowerState** column, then drop unwanted or redundant columns like **State** and **City**:

In [21]:
# Rename and drop redundant columns
loan_df.BorrowerState = loan_df.City
loan_df.drop(columns = ['State', 'City'], inplace=True)

In [22]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83982 entries, 0 to 83981
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ListingCreationDate  83982 non-null  datetime64[ns]
 1   ListingCategory      83982 non-null  object        
 2   BorrowerState        83982 non-null  object        
 3   IsBorrowerHomeowner  83982 non-null  bool          
 4   IncomeRange          83982 non-null  category      
 5   IncomeVerifiable     83982 non-null  bool          
 6   DebtToIncomeRatio    76768 non-null  float64       
 7   StatedMonthlyIncome  83982 non-null  float64       
 8   Term                 83982 non-null  int64         
 9   ProsperRating        83982 non-null  category      
 10  EmploymentStatus     83982 non-null  object        
 11  LoanStatus           83982 non-null  object        
 12  LoanOriginalAmount   83982 non-null  int64         
 13  BorrowerAPR          83982 non-

## Exploratory Data Analysis
---
We will explore our data systematically by building univariate, bivariate and multivariate visualizations.

### A. Univariate Exploration